<a href="https://colab.research.google.com/github/standha/Surprise-Recommendation-Algorithm/blob/main/surprise_%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.2 MB/s eta 0:00:00


In [26]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [115]:
import pandas as pd
import tensorflow as tf
import numpy as np

# 'tbl_post.csv'에서 'bno'와 'userid' 열을 선택
post_df = pd.read_csv('tbl_post.csv')
post_df['userid'] = post_df['userid'].astype(str)
post_ratings = post_df[['bno', 'userid']]

# 'tbl_board.csv'에서 'bno'와 'title' 열을 선택
board_df = pd.read_csv('tbl_board.csv')
board_df['title'] = board_df['title'].astype(str)
board_data = board_df[['bno', 'title']]

# 'bno'를 기준으로 두 데이터 프레임을 병합
merged_data = post_ratings.merge(board_data, on='bno', how='inner')

# # 필요한 열 선택
ratings = merged_data[['title', 'userid']]
movies = board_data[['title']]


def pandas_to_dataset(dataframe, target_column_name):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_column_name)  # 타겟 열 선택
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    return dataset

# ratings 및 movies 데이터프레임을 TensorFlow Datasets로 변환
# Create 'ratings_dataset' with 1D tensors
# ratings_dataset = tf.data.Dataset.from_tensor_slices((merged_data['userid'], merged_data['title']))

# # Create 'movies_dataset' using the 'title' column from 'movies' DataFrame
# movies_dataset = tf.data.Dataset.from_tensor_slices(movies['title'])
ratings_dataset = pandas_to_dataset(ratings,'userid')
movies_dataset = pandas_to_dataset(movies, 'title')

# Create a function to convert a row to the desired format
def map_features(x,y):
    return {
        "title": x["title"],
        "userid": y
    }

# # Apply the mapping function to each row in the dataset
ratings_dataset = ratings_dataset.map(map_features)
movies_dataset = tf.data.Dataset.from_tensor_slices(movies['title'])
# movies_dataset = tf.data.Dataset.from_tensor_slices(movies['title']).batch(128)

# print(len(list(ratings_dataset)))
print(len(list(movies_dataset)))
len(list(movies_dataset))
# len(list(ratings_dataset))

1182


1182

In [116]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings_dataset.map(lambda x : x['userid']))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies_dataset)

In [117]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task



  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["userid"])
    movie_embeddings = self.movie_model(features["title"])

    return self.task(user_embeddings, movie_embeddings)


# class MovieLensModel(tfrs.Model):
#     def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
#         super(MovieLensModel, self).__init__()
#         self.user_model = user_model
#         self.movie_model = movie_model
#         self.task = task

#     def call(self, features):
#         # 구현하려는 연산을 여기에 추가
#         user_embeddings = self.user_model(features["userid"])
#         movie_embeddings = self.movie_model(features["title"])
#         return user_embeddings, movie_embeddings

#     def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#         user_embeddings, movie_embeddings = self.call(features)
#         return self.task(user_embeddings, movie_embeddings)



In [118]:

# ratings 데이터셋 확인
for sample in ratings_dataset.take(1):
    print(sample)


{'title': <tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x8c\x8c \xeb\xb0\x94\xea\xb2\x8c\xed\x8a\xb8'>, 'userid': <tf.Tensor: shape=(), dtype=string, numpy=b'20204012'>}


In [119]:
# Define user and movie models.
# user_model = tf.keras.Sequential([
#     user_ids_vocabulary,
#     tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
# ])
# movie_model = tf.keras.Sequential([
#     movie_titles_vocabulary,
#     tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
# ])

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(input_dim=user_ids_vocabulary.vocab_size(), output_dim=64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(input_dim=movie_titles_vocabulary.vocab_size(), output_dim=64)
])

# user_model = tf.keras.Sequential([
#     user_ids_vocabulary,
#     tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
# ])
# movie_model = tf.keras.Sequential([
#     movie_titles_vocabulary,
#     tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
# ])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies_dataset.batch(128).map(movie_model)
))



In [120]:
# ratings_dataset 출력
for features, label in ratings_dataset.take(5):
    print(f'Features: {features}, Label: {label}')

# movies_dataset 출력
for title in movies_dataset.take(5):
    print(f'Title: {title}')
    # movies_dataset의 모양과 데이터 타입 확인
    print(title.shape, title.dtype)

# ratings_dataset 출력
for userid in ratings_dataset.take(5):
    print(f'Userid: {userid}')




tf.print(title)
tf.print(userid)


Features: title, Label: userid
Features: title, Label: userid
Features: title, Label: userid
Features: title, Label: userid
Features: title, Label: userid
Title: b'\xeb\x82\x98\xeb\xac\xbc\xeb\xb9\x84\xeb\xb9\x94\xeb\xb0\xa5'
() <dtype: 'string'>
Title: b'\xec\x98\xa4\xea\xb3\xa1\xeb\xb0\xa5'
() <dtype: 'string'>
Title: b'\xec\x9e\xa1\xec\xb1\x84\xeb\xb0\xa5'
() <dtype: 'string'>
Title: b'\xec\xbd\xa9\xeb\x82\x98\xeb\xac\xbc\xeb\xb0\xa5'
() <dtype: 'string'>
Title: b'\xec\x95\xbd\xec\x8b\x9d'
() <dtype: 'string'>
Userid: {'title': <tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x8c\x8c \xeb\xb0\x94\xea\xb2\x8c\xed\x8a\xb8'>, 'userid': <tf.Tensor: shape=(), dtype=string, numpy=b'20204012'>}
Userid: {'title': <tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x8c\x8c \xeb\xb0\x94\xea\xb2\x8c\xed\x8a\xb8'>, 'userid': <tf.Tensor: shape=(), dtype=string, numpy=b'20204012'>}
Userid: {'title': <tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x8c\x8c \xeb\xb0\x94\xea\xb2\x8c\xed\x8a\xb8'>, 'use

In [177]:

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings_dataset.batch(4096), epochs=3)
userid = "2020412"
##해당 아이디가 추천을 남긴 게시글 제외
# def get_rated_movies(userid, ratings_dataset):
#     return list(ratings_dataset[ratings_dataset['userid'] == userid]['title'])
# user_rated_movies = get_rated_movies(userid,ratings_dataset)
# all_movie_titles = list(movies_dataset['title'])
# recommended_movies = [title for title in all_movie_titles if title not in user_rated_movies]

# merged_data에서 받아온 userid가 작성한 title을 가져옵니다.
user_titles = merged_data[merged_data['userid'] == userid]['title'].unique()

# movies_dataset을 필터링하여 user_titles와 동일하지 않은 title만 남깁니다.
movies_dataset = movies_dataset.filter(lambda title: tf.math.logical_not(tf.math.reduce_any(tf.math.equal(title, user_titles))))


# movies 데이터프레임에서 해당 title을 제외합니다.
# movies = movies[~movies['title'].isin(user_titles)]



index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies_dataset.batch(100).map(lambda title: (title, model.movie_model(title))))


# Get some recommendations.
_, titles = index(np.array([userid], dtype=object))
titles = [title.numpy().decode('utf-8') for title in titles[0, :5]]
print(f"Top 3 recommendations for user {userid}: {titles}")



Epoch 1/3
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.7692 - factorized_top_k/top_5_categorical_accuracy: 0.9615 - factorized_top_k/top_10_categorical_accuracy: 0.9615 - factorized_top_k/top_50_categorical_accuracy: 0.9615 - factorized_top_k/top_100_categorical_accuracy: 0.9615 - loss: 125.9855 - regularization_loss: 0.0000e+00 - total_loss: 125.9855
Epoch 2/3
1/1 [==============================] - 0s 144ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0769 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 495.5263 - regularization_loss: 0.0000e+00 - total_loss: 495.5263
Epoch 3/3
1/1 [==============================] - 0s 72ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9231 - facto

In [165]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings_dataset.batch(4096), epochs=3)

# 사용자 아이디
userid = "csy4012"

# 해당 아이디가 후기를 남긴 title 가져오는 함수
def get_rated_movies(userid, ratings_dataset):
    user_rated_movies = []
    for element in ratings_dataset.as_numpy_iterator():
        if element['userid'] == userid:
            user_rated_movies.append(element['title'].decode('utf-8'))
    return user_rated_movies

# 사용자가 이미 평가한 영화 가져오기
user_rated_movies = get_rated_movies(userid, ratings_dataset)

# 모든 영화 title 가져오기
all_movie_titles = list(movies['title'])

# 추천을 받아오려는 사용자의 아이디를 제외하고 추천 목록 생성
recommended_movies = [title for title in all_movie_titles if title != userid and title not in user_rated_movies]

# Get some recommendations.
_, titles = index(np.array([userid], dtype=object))
titles = [title.numpy().decode('utf-8') for title in titles[0, :3]]
print(f"Top 3 recommendations for user {userid}: {titles}")



Epoch 1/3
1/1 [==============================] - 3s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9231 - factorized_top_k/top_10_categorical_accuracy: 0.9231 - factorized_top_k/top_50_categorical_accuracy: 0.9231 - factorized_top_k/top_100_categorical_accuracy: 0.9231 - loss: 100.1339 - regularization_loss: 0.0000e+00 - total_loss: 100.1339
Epoch 2/3
1/1 [==============================] - 0s 73ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9615 - factorized_top_k/top_10_categorical_accuracy: 0.9615 - factorized_top_k/top_50_categorical_accuracy: 0.9615 - factorized_top_k/top_100_categorical_accuracy: 0.9615 - loss: 123.9682 - regularization_loss: 0.0000e+00 - total_loss: 123.9682
Epoch 3/3
1/1 [==============================] - 0s 80ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0769 - factorized_top_k/top_5_categorical_accuracy: 0.9615 - fa

In [144]:
encoded_text = b'\xeb\x82\x99\xec\xa7\x80\xeb\xb3\xb6\xec\x9d\x8c'
decoded_text = encoded_text.decode('utf-8')
print(decoded_text)

낙지볶음


In [ ]:
# import mysql.connector
# import requests
# import json
# from datetime import datetime as datetime
# from bs4 import BeautifulSoup

# # 함수를 통해 음식 정보 크롤링하여 DB에 저장
# def save_food_info_to_db(name, ingredients, recipe):
#     try:
#         # 데이터베이스 연결
#         conn = mysql.connect(
#             user="kawa",
#             password="0000",
#             host="localhost",
#             port=3307,
#             database="board"
#         )
#         # 커서 생성
#         cursor = conn.cursor()

#         #날짜 생성
#         now = datetime.now()
#         date = now.strftime('%Y-%m-%d %H:%M:%S')

#         name = name
#         ingredients = ingredients
#         recipe = recipe

# #         cursor.execute('SELECT bno FROM tbl_board')
# #         id = cursor.fetchone()+1


#         # SQL 쿼리 실행하여 데이터베이스에 저장
#         cursor.execute(
#             f"INSERT INTO tbl_board(bno, title, content, ingredients, writer, regDate, viewCnt, likeCnt, replyCnt, imgPath, status, category, level, averagerating) VALUES ('26', \"{name}\", \"{recipe}\", \"{ingredients}\", '7', \"{date}\", 0, 0, 0, 'Fooriend.png', 'Y', '한식', '비건입문자', '2.5')"
#         )


#         # 변경사항 커밋
#         conn.commit()

#     except mysql.Error as e:
#         print(f"An error occurred: {e}")

#     finally:
#         # 연결 종료
#         if conn:
#             conn.close()

In [ ]:
import mysql.connector

# MariaDB 연결 정보 설정
host = 'localhost'  # 호스트 정보
user = 'kawa'       # 사용자 이름
password = '0000'   # 암호
database = 'board'  # 데이터베이스 이름
port = 3307         # 포트 번호

try:
  # 연결 생성
  conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    port=port
  )

# 커서 생성
  cursor = conn.cursor()

# SQL 쿼리 실행 예시
  query = "SELECT * FROM tbl_board"
  cursor.execute(query)

# 결과 가져오기
  result = cursor.fetchall()

# 연결 종료
  cursor.close()
  conn.close()
except mysql.connector.Error as e:
    # 예외가 발생하면 연결 실패 메시지를 출력
    print(f"연결 실패: {e}")

연결 실패: 2003: Can't connect to MySQL server on 'localhost:3307' (111 Connection refused)


In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 38.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [ ]:
!pip install protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.1.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install mysql-connector-python==8.0.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: mysql-connector-python
    Found existing installation: mysql-connector-python 8.1.0
    Uninstalling mysql-connector-python-8.1.0:
      Successfully uninstalled mysql-connector-python-8.1.0
